In [16]:
# Genetic Algorithm With parameters
# Note this if checking if the optimisation still works with larger range parameters
import numpy as np

# Parameters
population_size = 1000   # Number of individuals in each generation
num_variables = 6        # Number of parameters to optimize
num_parents = 500        # Number of parents selected to breed
num_generations = 100   # Number of generations
crossover_rate = 0.8     # Probability of crossover
mutation_rate = 0.01      # Probability of mutation
mutation_range = [10, 5E5, 0.05, 10E5, 50E-7, 50E-7]       # Range of mutation This has to be adjusted for test solution
lower_bounds = [1, 10E5, 0.001, 0.1E-6, 10E-6, 10E-6]    # Bounds on parameters
upper_bounds = [1001, 500E6, 0.21, 5E6, 100E-6, 300E-6]
max_values = np.array([1000, 500E6, 0.21, 3.5E6, 500E-6, 500E-6])  # For normilisation of the fitness function

# Test Solution
#solution = np.array([280, 250E6, 0.12, 3.5E6, 30E-6, 250E-6])       #Pit 
#solution = np.array([160, 268.45E6, 0.0395, 1E6, 48E-6, 250E-6])    #Burn back 
solution = np.array([205,250E6, 0.015, 1E-6, 80E-6, 5E-6])           #AFC Note: the tau and amplitude f_0 parameters have been switched in the same order as above
#solution = np.array([])


# Initialise population. Randomising different solutions 
no_pulses = np.random.randint(1, 1001, (population_size, 1))
f_0 = np.random.randint(10E5, 500E6, (population_size, 1))
amplitude = np.random.uniform(0.01, 0.21, (population_size, 1))
freq_sweep = np.random.uniform(0.1E-6, 5E6, (population_size, 1))
duration = np.random.uniform(10E-6, 500E-6, (population_size, 1))
delay = np.random.uniform(10E-6, 500E-6, (population_size, 1))
# Combining the parameters together 
population = np.hstack((no_pulses, f_0, amplitude, freq_sweep, duration, delay))

# Fitness function Normailises the parameters then compares the population to the solution. The closer the match the better the fitness.
def calculate_fitness(population, solution, max_values):
    # Normalize the solution and population using the max_values
    normalized_solution = solution / max_values
    normalized_population = population / max_values
    
    # Calculate the difference
    difference = 1 - np.abs(normalized_solution - normalized_population)
    
    # Calculate fitness as the mean of the differences
    fitness = np.mean(difference, axis=1)

    return fitness

# Crossover function randomly picks two individuals from the population.
# Then there is a probability that they crossover if so a random point is picked where they switch parameters and create two offspring
# i.e parent 1 [a a a a a] parent 2 [b b b b b] offspring [a a b b b], [b b a a a]
# If they dont crossover the parents become the offspring
def crossover(parents, crossover_rate, population_size):
    num_parents = parents.shape[0]
    num_variables = parents.shape[1]

    # Initialize offspring population array
    offspring_population = np.zeros((population_size, num_variables))
    current_count = 0

    while current_count < population_size:
        # Randomly shuffle the order of parents
        shuffled_indices = np.random.permutation(num_parents)
        shuffled_parents = parents[shuffled_indices, :]

        # Iterate over pairs of parents
        for i in range(0, num_parents, 2):
            if current_count >= population_size:
                break

            if i == num_parents - 1:
                offspring_population[current_count, :] = shuffled_parents[i, :]  # if odd number parents
                current_count += 1
            else:
                parent1 = shuffled_parents[i, :]
                parent2 = shuffled_parents[i + 1, :]

                if np.random.rand() < crossover_rate:
                    # Perform crossover
                    crossover_point = np.random.randint(1, num_variables)
                    offspring1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
                    offspring2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
                else:
                    # No crossover, offspring are copies of parents
                    offspring1 = parent1
                    offspring2 = parent2

                # Add offspring to the population
                offspring_population[current_count, :] = offspring1
                current_count += 1

                if current_count < population_size:
                    offspring_population[current_count, :] = offspring2
                    current_count += 1

    return offspring_population

# Mutation function. There is a probability that the individual can mutate (for every parameter) 
# If it does mutate there is a specified range it can mutate by 
def mutate(new_population, mutation_rate, mutation_range):
    population_size = new_population.shape[0]
    num_variables = new_population.shape[1]

    mutated_population = new_population.copy()

    for i in range(population_size):
        for j in range(num_variables):
            if np.random.rand() < mutation_rate:
                mutation_value = (np.random.rand() - 0.5) * 2 * mutation_range[j]
                mutated_population[i, j] = mutated_population[i, j] + mutation_value
                mutated_population[i, j] = max(lower_bounds[j], min(upper_bounds[j], mutated_population[i, j]))

    return mutated_population

# Genetic Algorithm. loops for specified number of generations 
# Process:
# Step 1: Random population is created
# Step 2: The fitness is of each individual is calculated
# Step 3: Individuals are randomly picked weighted by their fitness, for breeding
# Step 4: Randomly breeding the individuals where they can crossover dna (parameters) and can mutate
# Step 5: Repeat this process with the newly made population 
for i in range(num_generations):
    # Fitness
    fitness = calculate_fitness(population, solution, max_values)
    if np.max(fitness) >= 0.99:
        break
    # Selection. This randomly picks a specified number of individuals from the population to become the breeders of the next gen
    # Individuals with a better fitness have a higher probability of being picked 
    parents = population[np.random.choice(population_size, num_parents, p=fitness/fitness.sum())]

    # Breeding Crossover and Mutation
    population = crossover(parents, crossover_rate, population_size)
    population = mutate(population, mutation_rate, mutation_range)

# Evaluation
fitness = calculate_fitness(population, solution, max_values)
max_value = np.max(fitness)
index_of_max = np.argmax(fitness)
min_value = np.min(fitness)
index_of_min = np.argmin(fitness)
avg_fit = np.mean(fitness)
best_fit = max_value
best = population[index_of_max, :]
worse_fit = min_value
worse = population[index_of_min, :]

print("Average Fitness:", avg_fit)
print("Best Fitness:", best_fit)
print("Number of generations:", i+1)
#print("solution", solution)
#print("Best Solution:", best)
#print("Worst Fitness:", worse_fit)
#print("Worst Solution:", worse)


Average Fitness: 0.9376855068924365
Best Fitness: 0.9816753435558715
Number of generations: 100
